In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [25]:
driver = webdriver.Chrome()

In [26]:
url = 'https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&tab=main'
driver.get(url)

In [31]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.main_prodlist.main_prodlist_list > ul > li')
len(lis)

44

In [32]:
li = lis[0]
name = li.select_one('.click_log_product_standard_title_').get_text().strip()
name

'삼성전자 비스포크 제트 VS20A956A3'

In [34]:
spec = li.select_one('.intro_text').get_text().strip()
spec

'배터리 1개+물걸레, 침구, 펫브러쉬 제외 액세서리 4종 구성 (청소기+청정스테이션 합포장)'

In [58]:
price = li.select_one('.click_log_product_standard_price_').get_text().strip( )
price

'458,920원'

In [59]:
lines = []
for il in lis:
    name = li.select_one('.click_log_product_standard_title_').get_text().strip()
    spec = li.select_one('.intro_text').get_text().strip()
    price = li.select_one('.click_log_product_standard_price_').get_text().strip( )
    lines.append([name, spec, price])

## 두번째 페이지로 이동

In [60]:
driver.find_element(By.XPATH, '//*[@id="paginationArea"]/div/div/a[2]').click()

In [61]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.paging_number_wrap > a')
len(lis)

10

## 전체

In [ ]:
# 검색결과 건수를 가져오기
strong = soup.select('.stations-local-after > p > strong')
strong[1].get_text()

'35건'

In [ ]:
count = int(strong[1].get_text().replace('건', '').strip())
count

35

In [ ]:
import math
count = math.ceil(count/10)
count

4

# 전체 페이지

In [ ]:
driver.get(url)
time.sleep(1)
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [ ]:
import math
strong = soup.select('.stations-local-after > p > strong')
count = int(strong[1].get_text().replace('건', '').strip())
count = math.ceil(count/10)

In [ ]:
lines = []
for i in range(count):
    if i >= 1 and i%2 == 0:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(2)
    if i >= 1 and i%2 == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.find('address').string.strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append([name, addr, tel])    

In [ ]:
len(lines)

100

In [ ]:
df = pd.DataFrame(lines, columns = ['관서명', '주소', '전화번호'])
df

,관서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119
...,...,...,...
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119


In [ ]:
len(df['관서명'].unique())

100

In [ ]:
df.to_csv('서울소방서.csv', index=False)

##  카카오 api 사용해서 위도경도 가져오기

In [ ]:
import requests
from urllib.parse import quote

In [ ]:
with open ('kakao.txt') as file:
    kakao_key = file.read()

In [ ]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization':f'KakaoAK {kakao_key}'}

In [ ]:
# 반복문을 통해서 모든 소방청 위도 경도 좌표 구하기
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    try: 
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

은평소방서-수색-119 안전센터


In [ ]:
df[df.관서명.isin(['은평소방서-수색-119 안전센터'])].주소

30    서울특별시 은평구 수색로 294 (수색동)
Name: 주소, dtype: object

In [ ]:
url = f'{local_url}?query={quote("서울특별시 은평구 수색로 294")}'
result = requests.get(url, headers=header).json()
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [ ]:
df[df.관서명.isin(['은평소방서'])].주소

5    서울특별시 은평구 통일로 962(진관동)
Name: 주소, dtype: object

In [ ]:
df.주소[30] = '서울특별시 은평구 통일로 962'

In [ ]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    try: 
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

In [ ]:
import folium

In [ ]:
map  = folium.Map(location =[37.5502, 126.982], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        tooltip = f'{df.관서명[i]}',
        popup = folium.Popup(df.주소[i], max_width=200),
        icon = folium.Icon(icon = 'glyphicon glyphicon-home')
    ).add_to(map)
title = '<h3 align="center" style="font-size:20px> 서울소재 경찰서위치</h3>'
map.get_root().html.add_child(folium.Element(title))
map